---
description: This module provides functions that convert FEEMSResult to proto buffer
  message
output-file: convertfeemsresulttoproto.html
title: Converter FEEMSResult to proto

---



In [14]:
# | default_exp convert_feems_result_to_proto
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# | include: false

from nbdev.showdoc import *

In [16]:
# | export
from functools import cached_property
from typing import Union, List, Dict, Optional, cast

from feems.components_model.component_electric import FuelCellSystem
from feems.fuel import FuelSpecifiedBy, GHGEmissions
from feems.types_for_feems import FEEMSResult, TypeComponent, EmissionType
from feems.system_model import (
    MechanicalPropulsionSystemWithElectricPowerSystem,
    ElectricPowerSystem,
    HybridPropulsionSystem,
    MechanicalPropulsionSystem,
    FEEMSResultForMachinerySystem,
)
import pandas as pd
import numpy as np
import MachSysS.feems_result_pb2 as proto
import MachSysS.gymir_result_pb2 as proto_gymir
import logging

In [17]:
# | export
# Define logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
ch.setFormatter(formatter)

logger.addHandler(ch)

_COLUMN_NAMES = {
    "multi fuel consumption [kg]": "multi_fuel_consumption_kg",
    "electric energy consumption [MJ]": "electric_energy_consumption_mj",
    "mechanical energy consumption [MJ]": "mechanical_energy_consumption_mj",
    "energy_stored [MJ]": "energy_stored_mj",
    "running hours [h]": "running_hours_h",
    "CO2 emission [kg]": "co2_emissions_kg",
    "NOx emission [kg]": "nox_emissions_kg",
    "component type": "component_type",
    "rated capacity": "rated_capacity",
    "rated capacity unit": "rated_capacity_unit",
    "switchboard id": "switchboard_id",
    "shaftline id": "shaftline_id",
}


class FEEMSResultConverter:
    feems_result: Union[FEEMSResult, FEEMSResultForMachinerySystem, None]
    time_series_input: Optional[proto_gymir.TimeSeriesResult] = None
    fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO

    _system_feems: Union[
        MechanicalPropulsionSystemWithElectricPowerSystem,
        ElectricPowerSystem,
        HybridPropulsionSystem,
        None,
    ] = None
    _time_series_data_for_electric_component: Union[
        List[Dict[str, proto.TimeSeriesResultForComponent]], None
    ] = None
    _time_series_data_for_mechanical_component: Union[
        List[Dict[str, proto.TimeSeriesResultForComponent]], None
    ] = None

    def __init__(
        self,
        feems_result: Union[FEEMSResult, FEEMSResultForMachinerySystem],
        system_feems: Union[
            MechanicalPropulsionSystemWithElectricPowerSystem,
            ElectricPowerSystem,
            HybridPropulsionSystem,
        ],
        time_series_input: Optional[proto_gymir.TimeSeriesResult] = None,
        fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO,
    ):
        self.feems_result = feems_result
        self.time_series_input = time_series_input
        self.system_feems = system_feems
        self.fuel_specified_by = fuel_specified_by

    @property
    def system_feems(self):
        return self._system_feems

    @system_feems.setter
    def system_feems(self, value):
        self._system_feems = value
        self._time_series_data_for_electric_component = []
        self._time_series_data_for_mechanical_component = []

    @property
    def isSystemElectric(self) -> bool:
        return isinstance(self.system_feems, ElectricPowerSystem)

    @property
    def electric_system(self) -> ElectricPowerSystem:
        return (
            self.system_feems
            if self.isSystemElectric
            else self.system_feems.electric_system
        )

    @property
    def mechanical_system(self) -> MechanicalPropulsionSystem:
        return (
            self.system_feems.mechanical_system if not self.isSystemElectric else None
        )

    @cached_property
    def _time_interval_to_time_array(self) -> np.ndarray:
        number_points = self.electric_system.power_sources[0].power_output.size
        if self.time_series_input is None:
            time_interval_s = self.electric_system.time_interval_s
            if np.isscalar(time_interval_s):
                return np.linspace(
                    0, (number_points - 1) * time_interval_s, number_points
                )
            else:
                return time_interval_s.cumsum()
        else:
            return np.array(
                [
                    power_instance.epoch_s
                    for power_instance in self.time_series_input.propulsion_power_timeseries
                ]
            )[:number_points]

    def _retrieve_time_series_data_from_components(self):
        """Retrieve time series data from components of power sources and energy storages and
        store them in a list of dictionaries. Each dictionary contains the name of the component,
        the data in the proto data type of TimeSeriesResultForComponent and the switchboard id.
        """
        # in the feems result
        power_sources_electric = (
            self.electric_system.power_sources
            + self.electric_system.energy_storage
            + self.electric_system.pti_pto
        )
        if self.mechanical_system is not None:
            power_sources_mechanical = self.mechanical_system.main_engines
        else:
            power_sources_mechanical = []

        for power_source in power_sources_electric:
            result_proto = proto.TimeSeriesResultForComponent(
                time=self._time_interval_to_time_array,
                power_output_kw=power_source.power_output.tolist(),
            )
            if power_source.type == TypeComponent.GENSET:
                genset_result = (
                    power_source.get_fuel_cons_load_bsfc_from_power_out_generator_kw(
                        fuel_specified_by=self.fuel_specified_by,
                    )
                )
                for fuel in genset_result.engine.fuel_flow_rate_kg_per_s.fuels:
                    result_proto.fuel_consumption_kg_per_s.fuels.append(
                        proto.FuelArray(
                            fuel_type=fuel.fuel_type.value,
                            fuel_origin=fuel.origin.value,
                            fuel_specified_by=fuel.fuel_specified_by.value,
                            mass_or_mass_fraction=fuel.mass_or_mass_fraction.tolist(),
                            lhv_mj_per_g=fuel.lhv_mj_per_g,
                        )
                    )
            elif power_source.type in [
                TypeComponent.FUEL_CELL_SYSTEM,
                TypeComponent.FUEL_CELL,
            ]:
                power_source = cast(FuelCellSystem, power_source)
                fuel_cell_result = power_source.get_fuel_cell_run_point(
                    power_source.power_output
                )
                for fuel in fuel_cell_result.fuel_flow_rate_kg_per_s.fuels:
                    result_proto.fuel_consumption_kg_per_s.fuels.append(
                        proto.FuelArray(
                            fuel_type=fuel.fuel_type.value,
                            fuel_origin=fuel.origin.value,
                            fuel_specified_by=fuel.fuel_specified_by.value,
                            mass_or_mass_fraction=fuel.mass_or_mass_fraction.tolist(),
                            lhv_mj_per_g=fuel.lhv_mj_per_g,
                        )
                    )
            elif power_source.type in [
                TypeComponent.BATTERY,
                TypeComponent.BATTERY_SYSTEM,
            ]:
                pass
            elif power_source.type == TypeComponent.PTI_PTO_SYSTEM:
                pass
            else:
                raise NotImplementedError(
                    f"Retrieving time series data for component type {power_source.type} "
                    f"is not implemented."
                )
            self._time_series_data_for_electric_component.append(
                dict(
                    name=power_source.name,
                    data=result_proto,
                    node_id=power_source.switchboard_id,
                )
            )
        for power_source in power_sources_mechanical:
            result_proto = proto.TimeSeriesResultForComponent(
                time=self._time_interval_to_time_array,
                power_output_kw=power_source.power_output.tolist(),
            )
            engine_run_result = power_source.get_engine_run_point_from_power_out_kw(
                power_source.power_output
            )
            for fuel in engine_run_result.fuel_flow_rate_kg_per_s.fuels:
                result_proto.fuel_consumption_kg_per_s.fuels.append(
                    proto.FuelArray(
                        fuel_type=fuel.fuel_type.value,
                        fuel_origin=fuel.origin.value,
                        fuel_specified_by=fuel.fuel_specified_by.value,
                        mass_or_mass_fraction=fuel.mass_or_mass_fraction.tolist(),
                    )
                )
            self._time_series_data_for_mechanical_component.append(
                dict(
                    name=power_source.name,
                    data=result_proto,
                    node_id=power_source.shaft_line_id,
                )
            )

    def _get_feems_result_proto_for_subsystem(
        self,
        feems_result: FEEMSResult,
        feems_system: Union[ElectricPowerSystem, MechanicalPropulsionSystem],
        include_time_series_for_components: bool = False,
        verbose: bool = False,
    ) -> proto.FeemsResult:
        """Convert the result of FEEMS calculation to the proto data type of FeemsResult.
        Args:
            feems_result: The result of FEEMS calculation.
            feems_system: The system for which the FEEMS calculation is performed.
        Returns:
            The result of FEEMS calculation in the proto data type of FeemsResult.
        """
        is_electric = isinstance(feems_system, ElectricPowerSystem)
        if not is_electric and not isinstance(feems_system, MechanicalPropulsionSystem):
            raise TypeError(
                f"feems_system must be either ElectricPowerSystem or MechanicalPropulsionSystem, "
                f"but {type(feems_system)} is given."
            )
        if include_time_series_for_components:
            self._retrieve_time_series_data_from_components()
            time_series_data = (
                self._time_series_data_for_electric_component
                if is_electric
                else self._time_series_data_for_mechanical_component
            )
        else:
            time_series_data = None
        result = proto.FeemsResult()
        for key, value in feems_result.__dict__.items():
            if key == "detail_result":
                continue
            if key == "multi_fuel_consumption_total_kg":
                for fuel in value.fuels:
                    result.multi_fuel_consumption_total_kg.fuels.append(
                        proto.FuelScalar(
                            fuel_type=fuel.fuel_type.value,
                            fuel_origin=fuel.origin.value,
                            fuel_specified_by=fuel.fuel_specified_by.value,
                            mass_or_mass_fraction=fuel.mass_or_mass_fraction,
                            lhv_mj_per_g=fuel.lhv_mj_per_g,
                        )
                    )
                continue
            if key == "total_emission_kg":
                result.nox_emission_total_kg = (
                    value[EmissionType.NOX] if value is not None else 0.0
                )
                continue
            if key == "co2_emission_total_kg":
                value = cast(GHGEmissions, value)
                result.co2_emission_total_kg.CopyFrom(
                    proto.GHGEmissions(
                        well_to_tank=value.well_to_tank_kg_or_gco2eq_per_gfuel,
                        tank_to_wake=value.tank_to_wake_kg_or_gco2eq_per_gfuel,
                        well_to_wake=value.well_to_wake_kg_or_gco2eq_per_gfuel,
                        tank_to_wake_without_slip=value.tank_to_wake_kg_or_gco2eq_per_gfuel_without_slip,
                        well_to_wake_without_slip=value.well_to_wake_without_slip_kg_or_gco2eq_per_gfuel,
                    )
                )
                continue
            if hasattr(result, key):
                if value is not None:
                    setattr(result, key, value)
                else:
                    if verbose:
                        logger.warning(f"Value for {key} is None.")
            else:
                if verbose:
                    logger.warning(
                        f"There is no matching key in Protobuf message for {key}"
                    )
        if feems_result.detail_result is not None:
            for (
                component_name,
                component_result,
            ) in feems_result.detail_result.iterrows():
                result_per_component = proto.ResultPerComponent(
                    component_name=component_name
                )
                for key, value in component_result.items():
                    key_result_per_component = _COLUMN_NAMES.get(key)
                    if hasattr(
                        result_per_component, key_result_per_component
                    ) or result_per_component.HasField(key_result_per_component):
                        if key_result_per_component == "multi_fuel_consumption_kg":
                            for fuel in value.fuels:
                                result_per_component.multi_fuel_consumption_kg.fuels.append(
                                    proto.FuelScalar(
                                        fuel_type=fuel.fuel_type.value,
                                        fuel_origin=fuel.origin.value,
                                        fuel_specified_by=fuel.fuel_specified_by.value,
                                        mass_or_mass_fraction=fuel.mass_or_mass_fraction,
                                        lhv_mj_per_g=fuel.lhv_mj_per_g,
                                    )
                                )
                            continue
                        if key_result_per_component == "co2_emissions_kg":
                            value = cast(GHGEmissions, value)
                            result_per_component.co2_emissions_kg.CopyFrom(
                                proto.GHGEmissions(
                                    well_to_tank=value.well_to_tank_kg_or_gco2eq_per_gfuel,
                                    tank_to_wake=value.tank_to_wake_kg_or_gco2eq_per_gfuel,
                                    well_to_wake=value.well_to_wake_kg_or_gco2eq_per_gfuel,
                                    well_to_wake_without_slip=value.well_to_wake_without_slip_kg_or_gco2eq_per_gfuel,
                                    tank_to_wake_without_slip=value.tank_to_wake_kg_or_gco2eq_per_gfuel_without_slip,
                                )
                            )
                            continue
                        if value is not None:
                            setattr(
                                result_per_component, key_result_per_component, value
                            )
                        else:
                            if verbose:
                                logger.warning(f"Value for {key} is None.")
                    else:
                        if verbose:
                            logger.warning(
                                f"There is no matching key in Protobuf message for {key} in "
                                f"{component_name}: {key_result_per_component}"
                            )
                if include_time_series_for_components:
                    try:
                        switchboard_shaft_line_id = (
                            component_result["switchboard id"]
                            if is_electric
                            else component_result["shaftline id"]
                        )
                        result_per_component.result_time_series.CopyFrom(
                            next(
                                filter(
                                    lambda time_series_item: (
                                        time_series_item["name"] == component_name
                                    )
                                    and (
                                        time_series_item["node_id"]
                                        == switchboard_shaft_line_id
                                    ),
                                    time_series_data,
                                )
                            ).get("data")
                        )
                    except StopIteration:
                        logger.warning(
                            f"No time-series data found for {component_name}"
                        )
                result.detailed_result.append(result_per_component)
        return result

    def get_feems_result_proto(
        self, include_time_series_for_components: bool = False, verbose: bool = False
    ) -> proto.FeemsResultForMachinerySystem:
        if self.isSystemElectric:
            feems_result_electric = self._get_feems_result_proto_for_subsystem(
                feems_result=self.feems_result,
                feems_system=self.electric_system,
                include_time_series_for_components=include_time_series_for_components,
                verbose=verbose,
            )
            return proto.FeemsResultForMachinerySystem(
                electric_system=feems_result_electric
            )
        else:
            feems_result_electric = self._get_feems_result_proto_for_subsystem(
                feems_result=self.feems_result.electric_system,
                feems_system=self.electric_system,
                include_time_series_for_components=include_time_series_for_components,
                verbose=verbose,
            )
            feems_result_mechanical = self._get_feems_result_proto_for_subsystem(
                feems_result=self.feems_result.mechanical_system,
                feems_system=self.mechanical_system,
                include_time_series_for_components=include_time_series_for_components,
                verbose=verbose,
            )
            return proto.FeemsResultForMachinerySystem(
                electric_system=feems_result_electric,
                mechanical_system=feems_result_mechanical,
            )

    def get_timeseries_for_power_sources_and_energy_storage(self):
        df = pd.DataFrame()
        df.index = pd.Series(
            data=self._time_interval_to_time_array,
            name="time",
        )
        for each_time_series in [
            *self._time_series_data_for_electric_component,
            *self._time_series_data_for_mechanical_component,
        ]:
            name = each_time_series["name"]
            data = each_time_series["data"]
            df[f"{name}-power_output_kw"] = list(data.power_output_kw)
            if len(data.fuel_consumption_kg_per_s) > 0:
                df[f"{name}-fuel_consumption_kg_per_s"] = list(
                    data.fuel_consumption_kg_per_s
                )
        return df

In [18]:
import random
from MachSysS.gymir_result_pb2 import GymirResult, SimulationInstance
from MachSysS.system_structure_pb2 import Engine, FuelType
import os

from MachSysS.system_structure_pb2 import MachinerySystem
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems
from MachSysS.convert_feems_result_to_proto import FEEMSResultConverter
import numpy as np
from RunFeemsSim.machinery_calculation import MachineryCalculation

In [19]:
from feems.types_for_feems import TypePower


# Create a random GymirResult and load the system
def create_gymir_result() -> GymirResult:
    return GymirResult(
        name="test",
        auxiliary_load_kw=500 * random.random(),
        result=[
            SimulationInstance(epoch_s=100 * index + 1, power_kw=1000 * random.random())
            for index, _ in enumerate(range(10))
        ],
    )


path = os.path.join("tests", "system_proto.mss")
with open(path, "rb") as file:
    system_proto = MachinerySystem()
    system_proto.ParseFromString(file.read())


In [23]:
for each_fuel_specification in FuelSpecifiedBy:
    if each_fuel_specification in [FuelSpecifiedBy.NONE, FuelSpecifiedBy.USER]:
        continue
    print("Testing fuel specification: ", each_fuel_specification)  
    system_feems = convert_proto_propulsion_system_to_feems(system_proto)
    machinery_calculation = MachineryCalculation(
        feems_system=system_feems, maximum_allowed_power_source_load_percentage=80
    )
    gymir_result = create_gymir_result()
    res = machinery_calculation.calculate_machinery_system_output_from_gymir_result(
        gymir_result=gymir_result,
        fuel_specified_by=each_fuel_specification,
    )
    print(res)
    feems_result_converter = FEEMSResultConverter(
        feems_result=res, system_feems=machinery_calculation.system_feems
    )
    number_sources = len(system_feems.power_sources)
    number_batteries = len(system_feems.energy_storage)
    feems_result_proto = feems_result_converter.get_feems_result_proto(
        include_time_series_for_components=True
    )
    assert number_sources + number_batteries == len(
        feems_result_converter._time_series_data_for_electric_component
    )
    number_points = len(system_feems.power_sources[0].power_output)
    for each_time_series, component in zip(
        feems_result_converter._time_series_data_for_electric_component,
        system_feems.power_sources + system_feems.energy_storage,
    ):
        assert each_time_series.get("name") == component.name
        assert len(each_time_series.get("data").time) == number_points
        assert np.all(np.diff(each_time_series.get("data").time) > 0)
        assert len(each_time_series.get("data").power_output_kw) == number_points
        if component.power_type == TypePower.POWER_SOURCE:
            fuel_consumption = each_time_series.get("data").fuel_consumption_kg_per_s
            for fuel in fuel_consumption.fuels:
                assert len(fuel.mass_or_mass_fraction) == number_points
    print(feems_result_proto)
    feems_result_proto = feems_result_converter.get_feems_result_proto(
        include_time_series_for_components=False
    )
    print(feems_result_proto)



Testing fuel specification:  FuelSpecifiedBy.FUEL_EU_MARITIME
FEEMSResult(duration_s=np.float64(900.0), energy_consumption_electric_total_mj=0.0, energy_consumption_mechanical_total_mj=0.0, energy_stored_total_mj=np.float64(0.0), load_ratio_genset=None, running_hours_main_engines_hr=0.0, running_hours_genset_total_hr=np.float64(0.3611111111111111), running_hours_fuel_cell_total_hr=np.float64(0.0), running_hours_pti_pto_total_hr=0.0, total_emission_kg={<EmissionType.NOX: 2>: np.float64(0.39747939899032436)}, detail_result=                                          multi fuel consumption [kg]  \
Genset 1            FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Genset 2            FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Fuel cell system 1  FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Battery system 1                            FuelConsumption(fuels=[])   
Genset 3            FuelConsumption(fuels=[<feems.fuel.Fuel object...   
Genset 4            FuelConsumption

In [21]:
# Test for the case of no ICE in the system
import os
from MachSysS.utility import retrieve_machinery_system_from_file
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems
from MachSysS.gymir_result_pb2 import TimeSeriesResult
from RunFeemsSim.machinery_calculation import MachineryCalculation
import MachSysS.system_structure_pb2 as sys_structure_pb2
from MachSysS.convert_feems_result_to_proto import FEEMSResultConverter

# Import the system from the file
path = os.path.join("tests", "electric_propulsion_system.mss")
system_proto = retrieve_machinery_system_from_file(path)
system_proto_copy = sys_structure_pb2.MachinerySystem()
system_proto_copy.CopyFrom(system_proto)
# Remove genset from the system
for switchboard_idx, switchboard in enumerate(
    system_proto.electric_system.switchboards
):
    for subsystem in switchboard.subsystems:
        if subsystem.component_type == sys_structure_pb2.Subsystem.ComponentType.GENSET:
            print(f"removing genset: {subsystem.name}")
            system_proto_copy.electric_system.switchboards[
                switchboard_idx
            ].subsystems.remove(subsystem)
system_feems = convert_proto_propulsion_system_to_feems(system_proto_copy)
# Import the time series data
path = os.path.join("tests", "time_series_result.pb")
time_series_pb = TimeSeriesResult()
with open(path, "rb") as file:
    time_series_pb.ParseFromString(file.read())

# Create a machinery calculation object
machinery_calculation = MachineryCalculation(
    feems_system=system_feems, maximum_allowed_power_source_load_percentage=80
)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_pb,
)
res_converter = FEEMSResultConverter(
    feems_result=res, system_feems=system_feems, time_series_input=time_series_pb
)
res_proto = res_converter.get_feems_result_proto()
print(res_proto)

removing genset: Genset 1
removing genset: Genset 2
removing genset: Genset 3
removing genset: Genset 4
electric_system {
  duration_s: 10799940
  multi_fuel_consumption_total_kg {
    fuels {
      fuel_type: HYDROGEN
      fuel_origin: RENEWABLE_NON_BIO
      fuel_specified_by: IMO
      mass_or_mass_fraction: 4082989.3255475312
      lhv_mj_per_g: 0.12
    }
  }
  running_hours_fuel_cell_total_hr: 5268.75
  co2_emission_total_kg {
  }
  detailed_result {
    component_name: "Fuel cell system 1"
    multi_fuel_consumption_kg {
      fuels {
        fuel_type: HYDROGEN
        fuel_origin: RENEWABLE_NON_BIO
        fuel_specified_by: IMO
        mass_or_mass_fraction: 2029071.5767343191
        lhv_mj_per_g: 0.12
      }
    }
    running_hours_h: 2268.7666666666669
    co2_emissions_kg {
    }
    component_type: "FUEL_CELL_SYSTEM"
    rated_capacity: 3150
    rated_capacity_unit: "kW"
    switchboard_id: 1
  }
  detailed_result {
    component_name: "Battery system 1"
    co2_emissi